# Workflow for microbiome and metabolomics analyses

In [8]:
#demultiplexing, sequence quality control and feature table construction was done using Qiita 
#Qiita Study ID 11259
#mass spectral files are accessible from the MassIVE repository accession ID MSV000083077

# Microbiome 16S analysis

In [ ]:
#convert Greengenes reference database hit fasta sequences to qza format
!qiime tools import \
    --type FeatureData[Sequence] \
    --input-path MicrobiomeData/reference-hit.seqs.fa \
    --output-path Data/reference-hit.seqs.qza

In [4]:
#generate a tree for phylogenetic diversity analyses!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences Data/reference-hit.seqs.qza \
    --o-alignment Data/aligned-rep-seqs.qza \
    --o-masked-alignment Data/masked-aligned-rep-seqs.qza \
    --o-tree Data/unrooted-tree.qza \
    --o-rooted-tree Data/rooted-tree.qza

Saved FeatureData[AlignedSequence] to: Data/aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: Data/masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


In [15]:
#unzip qza and convert biom to text file
!biom convert \
    -i Data/feature-table.biom \
    -o Decontamination/feature-table.txt \
    --to-tsv

In [20]:
#perform decontamination steps in R with 'decontam' package and then remove contaminant sequences from the feature table
!qiime feature-table filter-features \
    --i-table Data/table.qza \
    --m-metadata-file Decontamination/contaminant_features-to-filter-out.csv \
    --p-exclude-ids \
    --o-filtered-table Data/decontam.qza

#feature data summary
!qiime feature-table summarize \
    --i-table Data/decontam.qza \
    --o-visualization Data/decontam.qzv

#!qiime tools view Data/decontam.qzv

Saved FeatureTable[Frequency] to: Data/decontam.qza
Saved Visualization to: Data/decontam.qzv
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.
libva error: /usr/lib/x86_64-linux-gnu/dri/i965_drv_video.so init failed
[18051:18051:0100/000000.754008:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [ ]:
#filter samples and features 
!qiime feature-table filter-features \
    --i-table Data/decontam.qza \
    --p-min-samples 2 \
    --o-filtered-table Data/decontam-sample-min2-table.qza

!qiime feature-table filter-samples \
    --i-table decontam-sample-min2-table.qza \
    --p-min-features 10 \
    --o-filtered-table Data/decontam-sample-min2-feature10-table.qza

In [19]:
#filter blank samples
!qiime feature-table filter-samples \
    --i-table Data/decontam.qza \
    --m-metadata-file Data/samples-to-keep.csv \
    --o-filtered-table Data/sample-decontam-filtered-table.qza

Saved FeatureTable[Frequency] to: noblank-table.qza


### Alpha and beta diversity analysis

In [30]:
#Rarefied biom at 3200 reads per sample
qiime diversity core-metrics-phylogenetic \
    --i-phylogeny Data/rooted-tree.qza \
    --i-table Data/Decontam.qza \
    --p-sampling-depth 3200 \
    --m-metadata-file Data/metadata.csv \
    --output-dir Data/core-metrics-results

#DEICODE rpca and biplot
!qiime deicode rpca \
    --i-table Data/Decontam.qza \
    --p-min-feature-count 10 \
    --p-min-sample-count 500 \
    --o-biplot Data/deicode/ordination.qza \
    --o-distance-matrix Data/deicode/distance.qza

!qiime emperor biplot \
    --i-biplot Data/deicode/ordination.qza \
    --m-sample-metadata-file Data/metadata.csv \
    --m-feature-metadata-file Data/taxonomy.qza \
    --o-visualization Data/deicode/biplot.qzv \
    --p-number-of-features 8

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

### Taxonomic analysis

In [21]:
# download pre-trained classifier classifier for V4 region trained on the Greengenes 13_8 99% OTUs
! wget \
  -O Data/"gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2021.4/common/gg-13-8-99-515-806-nb-classifier.qza"

--2022-03-23 19:14:00--  https://data.qiime2.org/2021.4/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2021.4/common/gg-13-8-99-515-806-nb-classifier.qza [following]
--2022-03-23 19:14:00--  https://s3-us-west-2.amazonaws.com/qiime2-data/2021.4/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.152.64
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.152.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28289645 (27M) [application/x-www-form-urlencoded]
Saving to: ‘Data/gg-13-8-99-515-806-nb-classifier.qza’

Data/gg-13-8-99-515 100%[===================>]  26.98M  20.3MB/s    in 1.3s    

2022-03-23 19:14:01 (20.3 MB/s) - ‘Data/

In [29]:
!qiime feature-classifier classify-sklearn \
    --i-classifier Data/gg-13-8-99-515-806-nb-classifier.qza \
    --i-reads Data/reference-hit.seqs.qza \
    --o-classification Data/taxonomy.qza

!qiime metadata tabulate \
    --m-input-file Data/taxonomy.qza \
    --o-visualization Data/taxonomy.qzv

#taxonomy barplots
!qiime taxa barplot \
    --i-table Data/table.qza \
    --i-taxonomy Data/taxonomy.qza \
    --m-metadata-file Data/metadata.csv \
    --o-visualization Data/taxa-bar-plots.qzv

#decontaminated taxonomy bar plots
!qiime taxa barplot \
    --i-table Data/decontam.qza \
    --i-taxonomy Data/taxonomy.qza \
    --m-metadata-file Data/metadata.txt \
    --o-visualization Data/decontam-taxa-bar-plots.qzv

Saved Visualization to: Data/decontam-taxa-bar-plots.qzv


### Differential abundnace analysis - songbird

In [1]:
!qiime songbird multinomial \
    --i-table Data/sample-decontam-filtered-table.qza \
    --m-metadata-file Data/metadata.csv \
    --p-formula "age+sex+race+del8+C(bmicat_bditotalcat, Treatment('HL'))" \
    --p-epochs 10000 \
    --p-differential-prior 0.1 \
    --p-training-column Testing \
    --p-summary-interval 1 \
    --o-differentials Data/songbird/songbird-differentials.qza \
    --o-regression-stats Data/songbird/songbird-regression-stats.qza \
    --o-regression-biplot Data/songbird/songbird-regression-biplot.qza


Saved FeatureData[Differential] to: songbird-differentials.qza
Saved SampleData[SongbirdStats] to: songbird-regression-stats.qza
Saved PCoAResults % Properties('biplot') to: songbird-regression-biplot.qza


In [2]:
!qiime songbird summarize-single \
    --i-regression-stats Data/songbird/songbird-regression-stats.qza \
    --o-visualization Data/songbird/songbird-regression-stats.qzv

Saved Visualization to: songbird-regression-stats.qzv


### Compare with null model

In [3]:
!qiime songbird multinomial \
    --i-table sample-decontam-filtered-table.qza \
    --m-metadata-file Data/metadata.csv \
    --p-formula "1" \
    --p-epochs 10000 \
    --p-differential-prior 0.1 \
    --p-training-column Testing \
    --p-summary-interval 1 \
    --o-differentials Data/songbird/null-diff.qza \
    --o-regression-stats Data/songbird/null-stats.qza \
    --o-regression-biplot Data/songbird/null-biplot.qza

Saved FeatureData[Differential] to: null-diff.qza
Saved SampleData[SongbirdStats] to: null-stats.qza
Saved PCoAResults % Properties('biplot') to: null-biplot.qza


### Visualize the first model's regression stats and the null model's
### regression stats

In [4]:
!qiime songbird summarize-paired \
    --i-regression-stats Data/songbird/songbird-regression-stats.qza \
    --i-baseline-stats Data/songbird/null-stats.qza \
    --o-visualization Data/songbird/paired-summary.qzv

Saved Visualization to: paired-summary.qzv


In [5]:
!qiime qurro differential-plot \
    --i-ranks Data/songbird/songbird-differentials.qza \
    --i-table Data/songbird/sample-decontam-filtered-table.qza \
    --m-sample-metadata-file Data/metadata.csv \
    --m-feature-metadata-file Data/taxonomy.qza \
    --o-visualization Data/songbird/qurro-plot.qzv

Saved Visualization to: qurro-plot.qzv


### Empress plot - interactive analyses of differential microbial features

In [ ]:
!qiime empress community-plot \
    --i-tree Data/rooted-tree.qza \
    --i-feature-table Data/sample-decontam-filtered-table.qza \
    --m-sample-metadata-file Data/metadata.csv \
    --m-feature-metadata-file Data/taxonomy.qza \
    --m-feature-metadata-file Data/empress/feature_importance.qza \
    --o-visualization Data/empress/empress.qzv

### Additional analysis - longitudinal analysis
### Compositional Tensor Factorization (CTF) for longitudinal measures of microbiome data - collection time of the day

In [ ]:
#Gemelli
!qiime gemelli ctf \
    --i-table Data/sample-decontam-filtered-table.qza \
    --m-sample-metadata-file Data/metadata-Hong3.csv \
    --m-feature-metadata-file Data/taxonomy.qza \
    --p-state-column sample_collection_time_hongcode \
    --p-individual-id-column anonymized_name \
    --output-dir Data/ctf-results

#biplot
qiime emperor biplot \
    --i-biplot Data/ctf-results/subject_biplot.qza \
    --m-sample-metadata-file Data/metadata.csv \
    --m-feature-metadata-file Data/taxonomy.qza \
    --p-number-of-features 100 \
    --o-visualization Data/ctf-results/subject_biplot.qzv


# Metabolomics analysis
### After MS1 feature finding and data processing using MZmine, run feature based mass spectral molecular networking (FBMN) analysis on GNPS 
### GNPS analysis ID: https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=f192a0030f694224a0ba8f08223a1323

In [ ]:
#Batch effect correction - remove features that are differentially abundant between the batches from ion intensity file
perl -nE 'BEGIN {$/="END IONS\n"} s/\n/\t/g; s/^\s+//; s/\s+$//; say'  MetabolomicsData/GNPS.txt > MetabolomicsData/intensities.tab
fgrep -v -f MetabolomicsData/feature_ids.csv MetabolomicsData/intensities.tab > MetabolomicsData/filtered_intensities.tab
perl -pE 's/\n/\n\n/; s/\t/\n/g' MetabolomicsData/filtered_intensities.tab > MetabolomicsData/filtered_intensities.txt

#Remove batch specific features from the MS quantification table
awk -F ',' 'NR==FNR {id[$1]; next} $1 in id' MetabolomicsData/ids.csv MetabolomicsData/table.csv > MetabolomicsData/quant_table.csv

In [ ]:
#microbe-metabolite interactions through MMvec
#use paired samples
!qiime mmvec paired-omics \
    --i-microbes MicrobeMetabolite/sample-decontam-filtered-table.qza \
    --i-metabolites MicrobeMetabolite/annotated-metabolites-table.qza \
    --output-dir metabolomics

!qiime metadata tabulate \
    --m-input-file MicrobeMetabolite/conditionals.qza \
    --o-visualization MicrobeMetabolite/conditionals-viz.qzv

!qiime emperor biplot \
    --i-biplot MicrobeMetabolite/conditional_biplot.qza \
    --m-sample-metadata-file MicrobeMetabolite/metabolite-metadata.tsv \
    --m-feature-metadata-file MicrobeMetabolite/microbe-metadata.tsv \
    --o-visualization MicrobeMetabolite/emperor.qzv

In [ ]:
#Differential analysis through Songbird and Qurro visualization
!qiime songbird multinomial \
    --i-table MetabolomicsData/no_blank_qiime2_table.qza \
    --m-metadata-file MetabolomicsData/qiime2_metadata_no_blanks.tsv \
    --p-formula "ATTRIBUTE_age + ATTRIBUTE_sex + ATTRIBUTE_race + ATTRIBUTE_metabolomicsplatenumber + ATTRIBUTE_del8 + C(ATTRIBUTE_bmicat_bditotalcat, Treatment('HL'))" \
    --p-epochs 10000 \
    --p-differential-prior 0.1 \
    --p-summary-interval 1 \
    --o-differentials MetabolomicsData/songbird/songbird-differentials-metabolomics.qza \
    --o-regression-stats MetabolomicsData/songbird/songbird-regression-stats-metabolomics.qza \
    --o-regression-biplot MetabolomicsData/songbird/songbird-regression-biplot-metabolomics.qza

qiime songbird summarize-single \
    --i-regression-stats MetabolomicsData/songbird/songbird-regression-stats-metabolomics.qza \
    --o-visualization MetabolomicsData/songbird/songbird-regression-stats-metabolomics.qzv

qiime qurro differential-plot \
    --i-ranks MetabolomicsData/songbird/songbird-differentials-metabolomics.qza \
    --i-table MetabolomicsData/no_blank_qiime2_table.qza \
    --m-sample-metadata-file MetabolomicsData/qiime2_metadata_no_blanks.tsv \
    --o-visualization MetabolomicsData/songbird/qurro-plot-metabolomics.qzv